# General Data Processing Notebook
-----------
Import the needed packages.

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from glob import glob
from rqpy import process, io
from time import time as tm
import matplotlib.pyplot as plt

## Define Paths

Define a list of paths to each series to be processed, as well as the channels (and corresponding detectors).

In [2]:
pathstotraces = [
                 '/path/to/raw/data/series/11111111_1111',
                ]

# set to True to save the RQs
lgcsave = True
savepath = "/path/to/save/rqs/"

# set to None to process all dumps
maxdumps = None

# choose the channels/detector that will be processed
channels = ['PFS1']
det = ['Z1']*len(channels)

fs = 625e3

## Load template

Specify the path to the pulse template/PSD. It is assumed that the user made the pulse template and PSD using the `make_template.ipynb` demo.

In [3]:
# update the path to the template
pathtotemplate = "/path/save/template/template_11111111_1111_v1.pkl"
with open(f'{pathtotemplate}', 'rb') as file:
    template_dict = pkl.load(file)

# extract the templates/psds (assuming they were made with make_template.ipynb)
psds = [template_dict[f'psd_{chan}{d}'] for chan, d in zip(channels, det)]
templates = [template_dict[f'template_{chan}{d}'] for chan, d in zip(channels, det)]

## Setup the RQ Settings

Use `rqpy.process.SetupRQ` to set up which RQs we want to calculate for all of the raw data. There are default values set so that the user could run this notebook without adjusting any of the RQ settings, but we show an example setup below.

In [4]:
setup = process.SetupRQ(templates, psds, fs, trigger=0)
setup.adjust_baseline(indbasepre=16000)
setup.adjust_integral(indstart=16000, 
                      indstop=18000)
setup.adjust_maxmin(use_min=False)
setup.adjust_ofamp_nodelay(calc_lowfreqchi2=True)
setup.adjust_ofamp_unconstrained(calc_lowfreqchi2=True, 
                                 pulse_direction_constraint=1)
setup.adjust_ofamp_constrained(nconstrain=100,
                               calc_lowfreqchi2=True,
                               pulse_direction_constraint=1)
setup.adjust_ofamp_pileup(lgcrun=False)
setup.adjust_chi2_lowfreq(fcutoff=16000)

## Loop Through Each Path, Calculating RQs

We now loop through all of the paths in `pathstotraces`, running the `rqpy.process.rq` function on each file. This function calculates the specified RQs from the `rqpy.process.SetupRQ` function and saves them to Pandas DataFrames as Pickle files for future analysis.

In [5]:
start = tm()
for ii, bundle in enumerate(pathstotraces):
    print(f'Processing bundle {ii}')
    
    if maxdumps is None:
        dumps = sorted(glob(f"{bundle}*.mid.gz"))
    else:
        dumps = sorted(glob(f"{bundle}*.mid.gz"))[:maxdumps]
        
    print(f"There are {len(dumps)} dumps.")
    series = bundle.rstrip('/').split('/')[-1]
    
    rq = process.rq(filelist=dumps, channels=channels, det=det, setup=setup, filetype="mid.gz",
                    lgcsavedumps=lgcsave, savepath=savepath, nprocess=1)
    
    if lgcsave:
        version = len(glob(f'{savepath}rq_df_{series}_v*.pkl')) + 1
        rq.to_pickle(f'{savepath}rq_df_{series}_v{version}.pkl')
        
print(tm()-start)

Processing bundle 0
There are 10 dumps.
On Series: 11111111_1111,  dump: F0001
On Series: 11111111_1111,  dump: F0002
On Series: 11111111_1111,  dump: F0003
On Series: 11111111_1111,  dump: F0004
On Series: 11111111_1111,  dump: F0005
On Series: 11111111_1111,  dump: F0006
On Series: 11111111_1111,  dump: F0007
On Series: 11111111_1111,  dump: F0008
On Series: 11111111_1111,  dump: F0009
On Series: 11111111_1111,  dump: F0010
252.10492968559265
